In [4]:
# autoreload
%load_ext autoreload
%autoreload 2

In [5]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.transpiler import Layout
from qiskit import QuantumCircuit, transpile
from mldec.hardware.topological_codes.circuits import RepetitionCode, HardwarePhaseFlipRepetitionCode, generate_initial_states, get_target_qubits
from mldec.utils.experiments_ibm import process_jobs

from qiskit_ibm_runtime.fake_provider import FakeTorino
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from mldec.datasets.reps_exp_rep_code_data import make_exp_dataset_name, save_data
from mldec.hardware.topological_codes.postprocessing import reshape_and_verify_correspondence
from mldec.hardware.topological_codes.postprocessing import convert_aer_to_sampler_format
from concurrent.futures import ProcessPoolExecutor
import os
import numpy as np


In [168]:
 # Set the number of qubits and rounds of syndrome measurement.
n = 5
T = 5

num_train = 4096 # training examples PER TRIAL, total training data is num_train * num_trials
num_val = 16384 # this is number of validation runs PERTRIAL

num_trials = 1 # every trial consists of a different initial state.
# delay_factors = [1, 1.1, 1.2, 1.3, 1.4, 2, 3, 4, 5]

num_repeats_arr = [0]
num_repeats_plus_one_arr = [x+1 for x in num_repeats_arr]


In [169]:
target_qubits = get_target_qubits(n)
device = FakeTorino()
noise_model = NoiseModel.from_backend(
    device, thermal_relaxation=True, gate_error=True, readout_error=True
)
backend = AerSimulator.from_backend(device, noise_model=noise_model, method='statevector')


In [180]:
# We will specify different initial qubit states, but due to bandwidth issues we run many
# shots per choice of initial state (same as Google 2023 paper).
train_initial_states, val_initial_states = generate_initial_states(n, num_trials)
train_initial_states = [[1, 0, 0, 0, 0]]
# build the list of circuits to execute
training_circuits = []
train_num_shots = []
tot_train = 0
for trial in range(num_trials):
    initial_state = train_initial_states[trial]
    for num_repeats_plus_one in num_repeats_plus_one_arr:
        # initialize a new 
        code = HardwarePhaseFlipRepetitionCode(n, T, initial_state=initial_state, 
                                               backend=backend, delay_factor=1, 
                                               target_qubits=target_qubits, num_repeats_plus_one=num_repeats_plus_one,
                                               resets=True)
        training_circuits.append(code.circuit)
        
        # display the circuit
        isa_wire_order = code.wire_order + list(range(2*n-1, len(code.circuit.qubits) ))
        display(code.circuit.draw(wire_order=isa_wire_order, fold=999))

        train_num_shots.append(num_train)
        tot_train += 1


┌───┐ ░ ┌─────────┐┌────┐┌─────────┐ ░ ┌─────────┐┌────┐┌─────────┐ ░     ░     ░ ┌─────────┐┌────┐┌─────────┐ ░              ░       ░ ┌─────────┐┌────┐┌─────────┐ ░     ░     ░ ┌─────────┐┌────┐┌─────────┐ ░              ░       ░ ┌─────────┐┌────┐┌─────────┐ ░     ░     ░ ┌─────────┐┌────┐┌─────────┐ ░              ░       ░ ┌─────────┐┌────┐┌─────────┐ ░     ░     ░ ┌─────────┐┌────┐┌─────────┐ ░              ░       ░ ┌─────────┐┌────┐┌─────────┐ ░     ░     ░ ┌─────────┐┌────┐┌─────────┐ ░              ░  ░ ┌───┐┌─┐            
      data_qubit_0_0: ┤ X ├─░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░─────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──────────────░───────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░─────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──────────────░───────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░─────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──────────────░───────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░─────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──────────────░───────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░─────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──────────────░──░─┤ H ├┤M├────────────
                      └───┘ ░ └─────────┘└────┘└─────────┘ ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░ ┌─┐          ░       ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░ ┌─┐          ░       ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░ ┌─┐          ░       ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░ ┌─┐          ░       ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░ ┌─┐          ░  ░ └───┘└╥┘            
   measure_qubit_0_0: ──────░──────────────────────────────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤M├──────────░──|0>──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤M├──────────░──|0>──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤M├──────────░──|0>──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤M├──────────░──|0>──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤M├──────────░──░───────╫─────────────
                            ░ ┌─────────┐┌────┐┌─────────┐ ░ ├─────────┤├────┤├─────────┤ ░     ░  │  ░ ├─────────┤├────┤├─────────┤ ░ └╥┘          ░       ░ ├─────────┤├────┤├─────────┤ ░     ░  │  ░ ├─────────┤├────┤├─────────┤ ░ └╥┘          ░       ░ ├─────────┤├────┤├─────────┤ ░     ░  │  ░ ├─────────┤├────┤├─────────┤ ░ └╥┘          ░       ░ ├─────────┤├────┤├─────────┤ ░     ░  │  ░ ├─────────┤├────┤├─────────┤ ░ └╥┘          ░       ░ ├─────────┤├────┤├─────────┤ ░     ░  │  ░ ├─────────┤├────┤├─────────┤ ░ └╥┘          ░  ░ ┌───┐ ║ ┌─┐         
      data_qubit_0_1: ──────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──╫───────────░───────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──╫───────────░───────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──╫───────────░───────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──╫───────────░───────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──■──░──■──░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──╫───────────░──░─┤ H ├─╫─┤M├─────────
                            ░ └─────────┘└────┘└─────────┘ ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░  ║ ┌─┐       ░       ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░  ║ ┌─┐       ░       ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░  ║ ┌─┐       ░       ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░  ║ ┌─┐       ░       ░ ├─────────┤├────┤├─────────┤ ░  │  ░     ░ ├─────────┤├────┤├─────────┤ ░  ║ ┌─┐       ░  ░ └───┘ ║ └╥┘         
   measure_qubit_0_1: ──────░──────────────────────────────░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) 

In [185]:
from qiskit.quantum_info import Statevector

transpiled_training_circuits = transpile(training_circuits, backend)

job_sim = backend.run(transpiled_training_circuits, shots=20)

train_job_result = job_sim.result()



In [186]:
train_job_result = convert_aer_to_sampler_format(train_job_result, transpiled_training_circuits)


In [187]:
out = process_jobs(n, T, train_job_result, train_job_result, num_repeats_plus_one_arr, train_initial_states, train_initial_states)

In [166]:
for m, (X, y) in out.items():
    # current shape: X: (n_trials, repetitions, n_data, n-1), y: (n_trials, n_data)
    X, y = reshape_and_verify_correspondence(X, y)
    print()
    for x in X:
        print(x)
        print()
    # new shapes: (n_trials*n_data, repetitions, n-1), y: (n_trials*n_data)
    break
    # save_data(X, y, fname)


[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]

[[0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0.

In [133]:
X_tr, Y_tr = out[1]

In [135]:
for x in X_tr:
    print(x)

[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 0. 0.]]]


In [64]:
train_initial_states

[[0, 0, 1, 0, 0]]

In [ ]:
from mldec.datasets.toy_problem_data import repetition_pcm

H = repetition_pcm(n)


array([1, 1, 0, 1], dtype=int32)

In [27]:
train_initial_states

[[1, 0, 1, 1, 0]]

In [24]:
result_sim

NameError: name 'result_sim' is not defined